In [1]:
# compare standalone mlp model performance to transfer learning
from sklearn.datasets.samples_generator import make_blobs
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from matplotlib import pyplot
from numpy import mean
from numpy import std

In [2]:
# prepare a blobs examples with a given random seed
def samples_for_seed(seed):
    # generate samples
    X, Y = make_blobs(n_samples=1000, centers=3, n_features=2, random_state=seed)
    # one hot encode output variable
    Y = to_categorical(Y)
    # split into train and test
    n_train = 500
    trainX, testX = X[:n_train, :], X[n_train:, :]
    trainY, testY = Y[:n_train], Y[n_train:]
    return trainX, trainY, testX, testY

In [3]:
# define and fit model on a training dataset
def fit_model(trainX, trainY):
    # define model
    model = Sequential()
    model.add(Dense(5, input_dim=2, activation='relu',
                   kernel_initializer='he_uniform'))
    model.add(Dense(5, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(3, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='sgd',
                 metrics=['accuracy'])
    # fit model
    model.fit(trainX, trainY, epochs=100, verbose=0)
    return model

In [4]:
# repeated evaluation of a standalone model
def eval_standalone_model(trainX, trainY, testX, testY, n_repeats):
    scores = list()
    for _ in range(n_repeats):
        # define and fit a new model on the train dataset
        model = fit_model(trainX, trainY)
        # evaluate model on test dataset
        _, test_acc = model.evaluate(testX, testY, verbose=0)
        scores.append(test_acc)
    return scores

In [5]:
# repeated evaluation of a model with transfer learning
def eval_transfer_model(trainX, trainY, testX, testY, n_fixed, n_repeats):
    scores = list()
    for _ in range(n_repeats):
        # load model
        model = load_model('model_1.h5')
        # mark layer weights as fixed or not trainable
        for i in range(n_fixed):
            model.layers[i].trainable = False
        # re-compile model
        model.compile(loss='categorical_crossentropy', optimizer='sgd',
                     metrics=['accuracy'])
        # fit model on train dataset
        model.fit(trainX, trainY, epochs=100, verbose=0)
        # evaluate model on test dataset
        _, test_acc = model.evaluate(testX, testY, verbose=0)
        scores.append(test_acc)
    return scores

In [6]:
# prepare data for problem 2
trainX, trainY, testX, testY = samples_for_seed(2)
n_repeats = 30
dists, dist_labels = list(), list()

In [7]:
# repeated evaluation of standalone model
standalone_scores = eval_standalone_model(trainX, trainY, testX, testY, n_repeats)
print('Standalone %.3f (%.3f)' % (mean(standalone_scores), std(standalone_scores)))
dists.append(standalone_scores)
dist_labels.append('standalone')

W1222 06:37:18.843013  6504 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1222 06:37:18.868916  6504 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1222 06:37:18.872416  6504 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1222 06:37:18.960206  6504 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1222 06:37:19.002522  6504 deprecation_wrapper.py:119] From C:\Users\Lenovo

Standalone 0.932 (0.050)


In [ ]:
# repeated evaluation of transfer learning model, vary fixed layers
n_fixed = 3
for i in range(n_fixed):
    scores = eval_transfer_model(trainX, trainY, testX, testY, i, n_repeats)
    print('Transfer (fixed=%s) %.3f (%.3f)' % (i, mean(scores), std(scores)))
    dists.append(scores)
    dist_labels.append('transfer f=' + str(i))

Transfer (fixed=0) 0.709 (0.016)


In [ ]:
# box and whisker plot of score distributions
pyplot.boxplot(dists, labels=dist_labels)
pyplot.show()